# Functions

These are the functions needed in order to run the extraction notebook. **Please do not edit this file as it will break the code**

In [1]:
import geopandas as gpd   # Note that you require geopandas version >= 0.7 that incluse clip see here for installation (https://gis.stackexchange.com/questions/360127/geopandas-0-6-1-installed-instead-of-0-7-0-in-conda-windows-10#)
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
from geojson import Feature, Point, FeatureCollection
import rasterio.fill
from shapely.geometry import shape, mapping
import json
#from earthpy import clip    clip has been deprecated to geopandas
import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

## Getting admin 1 boundary name to clusters

In [ ]:
def get_admin1_name(clusters, admin_col_name, crs):
    # Import layer
    messagebox.showinfo('OnSSET', 'Select the admin 1 boundaries')
    admin_1 = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    clusters_support = clusters
    
    #Project layer to proper crs
    admin_1_proj = admin_1.to_crs({ 'init': crs})
    
    # Apply spatial join 
    clusters = gpd.sjoin(clusters_support, admin_1_proj[["geometry", admin_col_name]], op='within').drop(['index_right'], axis=1)
    clusters.rename(columns = {admin_col_name:'Admin_1'}, inplace = True)
    
    #Return result
    return clusters

## Getting IDP & Refugee camps characteristics

In [ ]:
def get_IDPs_RefugeeCamps_status(clusters, camp_name, crs):
    # Import layer
    messagebox.showinfo('OnSSET', 'Select the layer of IDP')
    idp_gdf = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    clusters_support = clusters
    
    #Project layer to proper crs
    idp_gdf_proj = idp_gdf.to_crs({ 'init': crs})
    
    # Apply spatial join 
    cluster_support = gpd.sjoin(clusters, idp_gdf_proj[["geometry", camp_name]]).drop(['index_right'], axis=1)
    cluster_support["IDP_RefugeeCamp"] = np.where((cluster_support[camp_name].notnull()), 1, 0)
    clusters = pd.merge(clusters, cluster_support[['id', camp_name, "IDP_RefugeeCamp"]], on='id', how = 'left')
    clusters["IDP_RefugeeCamp"] = clusters["IDP_RefugeeCamp"].fillna(0)

    # Rename columns
    clusters.rename(columns = {camp_name:'Camp_name'}, inplace = True)
    
    #Return result
    return clusters

## Getting No of building per cluster

In [ ]:
def get_buildings_in_clusters(clusters, col_name, crs):
    # Import layer
    messagebox.showinfo('OnSSET', 'Select the layer of building footprints')
    gdf = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    
    #Converting polygon buildings to points
    gdf_centroids = gpd.GeoDataFrame(gdf,
                                     crs="EPSG:4326",
                                     geometry=[Point(xy) for xy in zip(gdf.centroid.x, gdf.centroid.y)])
    
    # Reverting clusters to original crs 
    clusters_support = clusters[['id', 'geometry']].to_crs({'init': "EPSG:4326"})
    #clusters_support.id = clusters_support.id.astype(int)
    
    # Apply spatial join and group by cluster "id"
    pointsInPolygon = gpd.sjoin(gdf_centroids, clusters_support, how="inner", op='intersects')
    pointsInPolygon[col_name]=1
    group_by_id = pointsInPolygon.groupby(["id"]).sum().reset_index().drop("index_right", axis=1)
    
    # Merge back to clusters
    clusters = pd.merge(clusters, group_by_id[['id', col_name]], on='id', how = 'left')
    
    # Fill NaN values with 0
    clusters[col_name] = clusters[col_name].fillna(0)
    
    #Return result
    return clusters

## Getting No of water points per cluster

In [ ]:
def get_waterpoints_in_clusters(clusters, col_name, crs):
    # Import layer
    messagebox.showinfo('OnSSET', 'Select the layer of water points')
    gdf = gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    
    # Reverting clusters to original crs 
    clusters_support = clusters[['id', 'geometry']].to_crs({'init': "EPSG:4326"})
    
    # Apply spatial join and group by cluster "id"
    pointsInPolygon = gpd.sjoin(gdf, clusters_support, how="inner", op='intersects')
    pointsInPolygon[col_name]=1
    group_by_id = pointsInPolygon.groupby(["id"]).sum().reset_index().drop("index_right", axis=1)
    
    # Merge back to clusters
    clusters = pd.merge(clusters, group_by_id[['id', col_name]], on='id', how = 'left')
    
    # Fill NaN values with 0
    clusters[col_name] = clusters[col_name].fillna(0)
    
    #Return result
    return clusters

## Processing Rasters

In [2]:
def processing_raster(name, method, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    raster=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=name, geojson_out=True, all_touched=True)
    
    print(datetime.datetime.now())
    return clusters

## Processing Elevation and Slope

In [3]:
def processing_elevation_and_slope(name, method, clusters, workspace,crs):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    raster=rasterio.open(filedialog.askopenfilename(filetypes = (("rasters","*.tif"),("all files","*.*"))))
    
    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=name, geojson_out=True, all_touched=True)

    gdal.Warp(workspace + r"\dem.tif",raster.name,dstSRS=crs)

    def calculate_slope(DEM):
        gdal.DEMProcessing(workspace + r'\slope.tif', DEM, 'slope')
        with rasterio.open(workspace + r'\slope.tif') as dataset:
            slope=dataset.read(1)
        return slope

    slope=calculate_slope(workspace + r"\dem.tif")

    slope = rasterio.open(workspace + r'\slope.tif')
    gdal.Warp(workspace + r'\slope_4326.tif',slope.name,dstSRS='EPSG:4326')
    slope_4326 = rasterio.open(workspace + r'\slope_4326.tif')

    clusters = zonal_stats(
        clusters,
        slope_4326.name,
        stats=["majority"],
        prefix="sl_", all_touched = True, geojson_out=True)
    
    print(datetime.datetime.now())
    return clusters

## Finalizing rasters

In [4]:
def finalizing_rasters(workspace, clusters, crs):
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(clusters)}
        dst.write(json.dumps(collection))
  
    clusters = gpd.read_file(output)
    os.remove(output)
    
    print(datetime.datetime.now())
    return clusters

## Preparing for vectors

In [5]:
def preparing_for_vectors(workspace, clusters, crs):   
    clusters.crs = {'init' :'epsg:4326'}
    clusters = clusters.to_crs({ 'init': crs}) 
    points = clusters.copy()
    points["geometry"] = points["geometry"].centroid
    points.to_file(workspace + r'\clusters_cp.shp', driver='ESRI Shapefile')
    print(datetime.datetime.now())    
    return clusters

## Processing Lines

In [6]:
def processing_lines(name, admin, crs, workspace, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    lines=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))

    lines_clip = gpd.clip(lines, admin)
    lines_clip.crs = {'init' :'epsg:4326'}
    lines_proj=lines_clip.to_crs({ 'init': crs})

    lines_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    line = fiona.open(workspace +  r"\ " + name + "_proj.shp")
    firstline = line.next()

    schema = {'geometry' : 'Point', 'properties' : {'id' : 'int'},}
    with fiona.open(workspace + r"\ " + name + "_proj_points.shp", "w", "ESRI Shapefile", schema) as output:
        for lines in line:
            if lines["geometry"] is not None:
                first = shape(lines['geometry'])
                length = first.length
                for distance in range(0,int(length),100):
                    point = first.interpolate(distance)
                    output.write({'geometry' :mapping(point), 'properties' : {'id':1}})

    lines_f = fiona.open(workspace + r"\ " + name + "_proj_points.shp")
    lines = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    points = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in lines_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)

    geoms2 = [shape(feat["geometry"]) for feat in points]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    results1, results2 = do_kdtree(s1_arr,s2_arr)

    z=results1.tolist()
    clusters[name+'Dist'] = z
    clusters[name+'Dist'] = clusters[name+'Dist']/1000

    a = vector_overlap(lines, clusters, name+'Dist')

    clusters = pd.merge(left = clusters, right = a[['id',name+'Dist2']], on='id', how = 'left')
    clusters.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    clusters.loc[clusters[name+'Dist2'] == 0, name+'Dist'] = 0

    del clusters[name+'Dist2']
    print(datetime.datetime.now())
    return clusters

## Processing points


In [3]:
def processing_points(name, admin, crs, workspace, clusters, mg_filter):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    points=gpd.read_file(filedialog.askopenfilename(filetypes = (("shapefile","*.shp"),("all files","*.*"))))
    if mg_filter:
        points['umgid'] = range(0, len(points))
        points_post = points

    points_clip = gpd.clip(points, admin)
    points_clip.crs = {'init' :'epsg:4326'}
    points_proj=points_clip.to_crs({ 'init': crs})

    points_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    points_f = fiona.open(workspace + r"\ " + name + "_proj.shp")
    points = gpd.read_file(workspace +  r"\ " + name + "_proj.shp")
    points2 = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in points_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)
    
    geoms2 = [shape(feat["geometry"]) for feat in points2]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays,points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    results1, results2 = do_kdtree(s1_arr,s2_arr)

    z=results1.tolist()
    clusters[name+'Dist'] = z
    clusters[name+'Dist'] = clusters[name+'Dist']/1000.
    if mg_filter:
        z2 = results2.tolist()
        clusters['umgid'] = z2

    a = vector_overlap(points, clusters, name+'Dist')

    clusters = pd.merge(left = clusters, right = a[['id',name+'Dist2']], on='id', how = 'left')
    clusters.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    clusters.loc[clusters[name+'Dist2'] == 0, name+'Dist'] = 0
    
    if mg_filter:
        clusters = pd.merge(clusters, points_post[['umgid', 'name', "MV_network", "MG_type"]], on='umgid', how = 'left')
        clusters.rename(columns = {'name':'ESP_name',
                                   'MV_network':'ESP_MV_status',
                                   'MG_type':'ESP_type'}, inplace = True)

    del clusters[name+'Dist2']
    del clusters['umgid']
    print(datetime.datetime.now())
    return clusters

## Processing hydro

In [8]:
def processing_hydro(admin, crs, workspace, clusters, points, hydropowervalue, 
                     hydropowerunit):

    points_clip = gpd.clip(points, admin)
    points_clip.crs = {'init' :'epsg:4326'}
    points_proj=points_clip.to_crs({ 'init': crs})

    points_proj.to_file(workspace + r"\HydropowerDist_proj.shp", driver='ESRI Shapefile')
    points_f = fiona.open(workspace +  r"\HydropowerDist_proj.shp")
    points = gpd.read_file(workspace +  r"\HydropowerDist_proj.shp")
    points2 = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in points_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)
    
    geoms2 = [shape(feat["geometry"]) for feat in points2]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    mytree = scipy.spatial.cKDTree(s1_arr)
    dist, indexes = mytree.query(s2_arr)
            
    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op = 'intersects')
        a[column_name + '2'] = 0
        return a  

    z1=dist.tolist()
    z2=indexes.tolist()
    clusters['HydropowerDist'] = z1
    clusters['HydropowerDist'] = clusters['HydropowerDist']/1000
    clusters['HydropowerFID'] = z2
    
    z3 = []
    for s in indexes:
        z3.append(points[hydropowervalue][s])
        
    clusters['Hydropower'] = z3
    
    x = hydropowerunit
    
    if x is 'MW':
        clusters['Hydropower'] = clusters['Hydropower']*1000
    elif x is 'kW':
        clusters['Hydropower'] = clusters['Hydropower']
    else:
        clusters['Hydropower'] = clusters['Hydropower']/1000

    a = vector_overlap(points, clusters, 'HydropowerDist')

    clusters = pd.merge(left = clusters, right = a[['id','HydropowerDist2']], on='id', how = 'left')
    clusters.drop_duplicates(subset ="id", keep = "first", inplace = True) 

    clusters.loc[clusters['HydropowerDist2'] == 0, 'HydropowerDist'] = 0

    del clusters['HydropowerDist2']
    print(datetime.datetime.now())
    return clusters

## Creating the prioritization columns for filter visualization

In [ ]:
def create_prio_columns(clusters):
    if "HF_kWh" in clusters:
        clusters["School"] = np.where((clusters["HF_kWh"] > 0), 1, 0)
    
    if "EF_kWh" in clusters:
        clusters["Health_facility"] = np.where((clusters["EF_kWh"] > 0), 1, 0)
    
    if "waterpoints_count" in clusters:
        clusters["Water_point"] = np.where((clusters["waterpoints_count"] > 0), 1, 0)
    
    return clusters

## Conditioning

In [9]:
def conditioning(clusters, workspace, popunit):
    clusters = clusters.to_crs({ 'init': 'epsg:4326'}) 

    clusters = clusters.rename(columns={"NightLight": "NightLights", popunit : "Pop",})

    if "Pop_cellscount" in clusters:
        clusters = clusters.rename(columns={"ClusterCellscount": "ClusterCells"})
        
    if "CoreCellscount" in clusters:
        clusters = clusters.rename(columns={"CoreCellscount": "CoreCells"})
        
    if "landcovermajority" in clusters:
        clusters = clusters.rename(columns={"landcovermajority": "LandCover"})

    if "elevationmean" in clusters:
        clusters = clusters.rename(columns={"elevationmean": "Elevation"})  

    if "sl_majority" in clusters:
        clusters = clusters.rename(columns={"sl_majority": "Slope"})

    if "ghimean" in clusters:
        clusters = clusters.rename(columns={"ghimean": "GHI"})
        
    if "traveltimemean" in clusters:
        clusters["traveltimemean"] = clusters["traveltimemean"]/60
        clusters = clusters.rename(columns={"traveltimemean": "TravelHours"})
    elif "TravelHour" in clusters:
        clusters = clusters.rename(columns={"TravelHour": "TravelHours"})
        
    if "windmean" in clusters:
        clusters = clusters.rename(columns={"windmean": "WindVel"})
    
    if "Residentia" in clusters:
        clusters = clusters.rename(columns={"Resudentia": "ResidentialDemandTierCustom"})
    elif "customdemandmean" in clusters:
        clusters = clusters.rename(columns={"customdemandmean": "ResidentialDemandTierCustom"})
    else:
        clusters["ResidentialDemandTierCustom"] = 0
    
    if "Substation" in clusters:
        clusers = clusters.rename(columns={"Substation": "SubstationDist"})
    elif "SubstationDist" not in clusters:
        clusters["SubstationDist"] = 99999

    if "CurrentHVL" in clusters:
        clusters = clusters.rename(columns={"CurrentHVL": "Existing_HVDist"})
    
    if "CurrentMVL" in clusters:
        clusters = clusters.rename(columns={"CurrentMVL": "Existing_MVDist"})
    
    if "PlannedHVL" in clusters:
        clusters = clusters.rename(columns={"PlannedHVL": "Planned_HVDist"})
    
    if "PlannedMVL" in clusters:
        clusters = clusters.rename(columns={"PlannedMVL": "Planned_MVDist"})

    if "Existing_HVDist" in clusters:
        clusters = clusters.rename(columns={"Existing_HVDist": "CurrentHVLineDist"})
        if "Planned_HVDist" in clusters:    
            mask = (clusters['Planned_HVDist'] > clusters['CurrentHVLineDist'])
            clusters['Planned_HVDist'][mask] = clusters['CurrentHVLineDist']
            clusters = clusters.rename(columns={"Planned_HVDist": "PlannedHVLineDist"})
        else:
            clusters["PlannedHVLineDist"] = clusters["CurrentHVLineDist"]
    elif "Existing_HVDist" not in clusters and "Planned_HVDist" not in clusters:
        clusters["PlannedHVLineDist"] = 99999
        clusters["CurrentHVLineDist"] = 99999
    else:
        clusters["CurrentHVLineDist"] = 99999
        clusters = clusters.rename(columns={"Planned_HVDist": "PlannedHVLineDist"})

    if "Existing_MVDist" in clusters:
        clusters = clusters.rename(columns={"Existing_MVDist": "CurrentMVLineDist"})
        if "Planned_MVDist" in clusters:    
            mask = (clusters['Planned_MVDist'] > clusters['CurrentMVLineDist'])
            clusters['Planned_MVDist'][mask] = clusters['CurrentMVLineDist']
            clusters = clusters.rename(columns={"Planned_MVDist": "PlannedMVLineDist"})
        else:
            clusters["PlannedMVLineDist"] = clusters["CurrentMVLineDist"]
    elif "Existing_MVDist" not in clusters and "Planned_MVDist" not in clusters:
        clusters["PlannedMVLineDist"] = 99999
        clusters["CurrentMVLineDist"] = 99999
    else:
        clusters["CurrentMVLineDist"] = 99999
        clusters = clusters.rename(columns={"Planned_MVDist": "PlannedMVLineDist"})

    if "RoadsDist" not in clusters:
        clusters["RoadsDist"] = 99999

    if "Transforme" in clusters: 
        clusters = clusters.rename(columns={"Transforme": "TransformerDist"})
    elif "TransformerDist" not in clusters:
        clusters["TransformerDist"] = 99999

    if "Hydropower" not in clusters:
        clusters["Hydropower"] = 0
        
    if "Hydropow_1" in clusters:
        clusters = clusters.rename(columns={"Hydropow_1": "HydropowerDist"})
    elif 'HydropowerDist' not in clusters:
        clusters["HydropowerDist"] = 99999
        
    if "Hydropow_2" in clusters:
        clusters = clusters.rename(columns={"Hydropow_2": "HydropowerFID"})
    elif "HydropowerFID" not in clusters:
        clusters["HydropowerFID"] = 0
    
    if "IsUrban" not in clusters:
        clusters["IsUrban"] = 0    
        
    if "PerCapitaD" not in clusters:
        clusters["PerCapitaDemand"] = 0
    else:
        clusters = clusters.rename(columns={"PerCapitaD": "PerCapitaDemand"})
        
    if "HealthDema" not in clusters:
        clusters["HealthDemand"] = 0     
    else:
        clusters = clusters.rename(columns={"HealthDema": "HealthDemand"})
        
    if "EducationD" not in clusters:
        clusters["EducationDemand"] = 0     
    else:
        clusters = clusters.rename(columns={"EducationD": "EducationDemand"})   
        
    if "AgriDemand" not in clusters:
        clusters["AgriDemand"] = 0  
        
    if "Commercial" not in clusters:
        clusters["CommercialDemand"] = 0
    else:
        clusters = clusters.rename(columns={"Commercial": "CommercialDemand"})
        
    if "Conflict" not in clusters:
        clusters["Conflict"] = 0       

    if "Electrific" not in clusters:
        clusters["ElectrificationOrder"] = 0
    else:
        clusters = clusters.rename(columns={"Electrific": "ElectrificationOrder"})
    
    if "Resident_1" not in clusters:
        clusters["ResidentialDemandTier1"] = 7.74
    else: 
        clusters = clusters.rename(columns={"Resident_1": "ResidentialDemandTier1"})

    if "Resident_2" not in clusters:
        clusters["ResidentialDemandTier2"] = 43.8
    else: 
        clusters = clusters.rename(columns={"Resident_2": "ResidentialDemandTier2"})

    if "Resident_3" not in clusters:
        clusters["ResidentialDemandTier3"] = 160.6
    else: 
        clusters = clusters.rename(columns={"Resident_3": "ResidentialDemandTier3"})

    if "Resident_4" not in clusters:
        clusters["ResidentialDemandTier4"] = 423.4
    else: 
        clusters = clusters.rename(columns={"Resident_4": "ResidentialDemandTier4"})
    
    if "Resident_5" not in clusters:
        clusters["ResidentialDemandTier5"] = 598.6
    else: 
        clusters = clusters.rename(columns={"Resident_5": "ResidentialDemandTier5"})
        
    if "Existing_ESP_Dist" not in clusters:
        clusters["Existing_ESP_Dist"] = 99999
    
    if "ESP_name" not in clusters:
        clusters["ESP_name"] = None
        
    if "ESP_MV_status" not in clusters:
        clusters["ESP_MV_status"] = None
        
    if "ESP_type" not in clusters:
        clusters["ESP_type"] = None
        
    if "waterpoints_count" in clusters:
        clusters = clusters.rename(columns={"waterpoints_count": "waterpoints"})
    
    clusters["X_deg"] = clusters.geometry.centroid.x
    
    clusters["Y_deg"] = clusters.geometry.centroid.y
    
    #clusters.to_file(workspace + r"\output.shp", driver='ESRI Shapefile')
    clusters.to_file(workspace + r"\output.csv", driver='CSV')
    print(datetime.datetime.now())
    return clusters